In [2]:
import pandas as pd
import numpy as np
import json
import requests

In [10]:
country_abbrevs = pd.read_csv('country_abbrev_list.csv')

In [11]:
re = requests.get('https://sandman.sleepcycle.com/data/per-country-stats')
country_sleep_data = pd.DataFrame(re.json())

In [15]:
country_sleep_data['country'] = country_sleep_data['country'].str.upper()

In [19]:
pd.merge(country_sleep_data, country_abbrevs, left_on = 'country', right_on = 'Code').drop(['country', 'Code'], axis=1).set_index('Name')

,avg_sleep_quality,avg_duration,avg_snore_duration,avg_bedtime,avg_wakeup,group_size
Name,,,,,,
Italy,0.7486682951500879,26559.266933393454,3169.7523479872552,0.006817214674656151,0.31448310741894825,17746
Switzerland,0.7676812535348072,26966.307348145903,2547.1999160720907,-0.012167234332564362,0.30000670905745863,13187
United Kingdom,0.780649686520754,27615.676629481506,2772.777680841947,-0.01186164402361364,0.3085906251711859,71265
Austria,0.763325385442557,26907.30985915492,2709.724184204566,-0.012648748895109788,0.29917471930840633,5041
Israel,0.7355701689051373,26065.960356788895,2240.8429112245526,0.019832502675134714,0.32190980373380546,2018
...,...,...,...,...,...,...
United States,0.7695099636787335,27199.082732336738,2523.8290978294563,-0.020008372214502374,0.2955721450702383,263174
Spain,0.74899044525664,26872.62806206668,2691.0476601343175,0.016721078312036328,0.3282552066486292,15145
Indonesia,0.7428848846657335,26440.474689589297,2389.171329142869,-0.008663822087112836,0.2968566622293189,1047
